In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import tensorflow as tf

tf.logging.set_verbosity(tf.logging.INFO)

In [23]:
data = np.loadtxt('concrete.txt')
np.random.shuffle(data)
cutoff = int(len(data)*0.8)
X_train, y_train  = data[:cutoff, :-1], data[:cutoff, -1]
X_test, y_test = data[cutoff:, :-1], data[cutoff:, -1]

In [24]:
std_X_train = np.std(X_train, 0)
std_X_train[std_X_train == 0] = 1
mean_X_train = np.mean(X_train, 0)
X_train = (X_train - np.full(X_train.shape, mean_X_train)) / np.full(X_train.shape, std_X_train)
X_test = (X_test - np.full(X_test.shape, mean_X_train)) / np.full(X_test.shape, std_X_train)

In [25]:
#mean_y_train = np.mean(y_train)
#std_y_train = np.std(y_train)

#y_train = (y_train - mean_y_train) / std_y_train
#y_train = np.array(y_train, ndmin = 2).T.flatten()

#y_test = (y_test - mean_y_train) / std_y_train

In [26]:
# Set model params
model_params = {"learning_rate": 0.001,
                "dropout": 0.01,
                "hidden_width": 50}

In [27]:
def model_fn(features, labels, mode, params):
    '''
    features: A dict containing the features passed to the model via input_fn.
    labels: A Tensor containing the labels passed to the model via input_fn. 
            Will be empty for predict() calls, as these are the values the model will infer.
    mode: One of the following tf.estimator.ModeKeys string values indicating the context in which 
          the model_fn was invoked
          
    Return: tf.estimator.EstimatorSpec object
    '''
    ### 1. Configure the model via TensorFlow operations
    
    # Connect the first hidden layer to input layer
    # (features["x"]) with relu activation
    
    dropout_1 = tf.layers.dropout(inputs=features["x"],
                            rate=params['dropout'],
                            training=True)
    
    hidden_layer = tf.layers.dense(dropout_1, 
                                         units=params['hidden_width'],
                                         activation=tf.nn.relu)

    dropout_2 = tf.layers.dropout(inputs=hidden_layer,
                                rate=params['dropout'],
                                training=True)

    output_layer = tf.layers.dense(dropout_2, 
                                   units=1)

    predictions = tf.reshape(output_layer, [-1])
    
    # Provide an estimator spec for `ModeKeys.PREDICT`.
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode,
                                          predictions={"y": predictions})
    
    onehot_labels = tf.one_hot(indices=tf.cast(labels, tf.int32), depth=10)
    
    loss = tf.losses.softmax_cross_entropy(onehot_labels=onehot_labels, logits=logits)
    
    ### 2. Define the loss function for training/evaluation
    loss = tf.losses.mean_squared_error(labels, predictions)
    eval_metric_ops = {"rmse": 
                       tf.metrics.root_mean_squared_error(tf.cast(labels, tf.float64), predictions)}
    
    ### 3. Define the training operation/optimizer
    optimizer = tf.train.AdamOptimizer(learning_rate=params["learning_rate"])
    train_op = optimizer.minimize(loss=loss, global_step=tf.train.get_global_step())
    
    
    ### 4. Return predictions/loss/train_op/eval_metric_ops in EstimatorSpec object
    
    return tf.estimator.EstimatorSpec(mode=mode,
                                      loss=loss,
                                      train_op=train_op,
                                      eval_metric_ops=eval_metric_ops)

In [28]:
# Instantiate Estimator
nn = tf.estimator.Estimator(model_fn=model_fn, params=model_params)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {}


In [29]:
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": X_train},
    y=y_train_normalized,
    batch_size=32,
    num_epochs=None,
    shuffle=False)

In [30]:
# Train
nn.train(input_fn=train_input_fn,steps=40*32)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/tmpc1j8auxb/model.ckpt.
INFO:tensorflow:loss = 1.48527, step = 1
INFO:tensorflow:global_step/sec: 694.986
INFO:tensorflow:loss = 1.27576, step = 101 (0.145 sec)
INFO:tensorflow:global_step/sec: 703.065
INFO:tensorflow:loss = 1.00183, step = 201 (0.142 sec)
INFO:tensorflow:global_step/sec: 760.804
INFO:tensorflow:loss = 1.03118, step = 301 (0.131 sec)
INFO:tensorflow:global_step/sec: 802.458
INFO:tensorflow:loss = 0.996618, step = 401 (0.125 sec)
INFO:tensorflow:global_step/sec: 791.044
INFO:tensorflow:loss = 1.1489, step = 501 (0.126 sec)
INFO:tensorflow:global_step/sec: 785.327
INFO:tensorflow:loss = 1.04687, step = 601 (0.127 sec)
INFO:tensorflow:global_step/sec: 807.938
INFO:tensorflow:loss = 0.86468, step = 701 (0.124 sec)
INFO:tensorflow:global_step/sec: 728.685
INFO:tensorflow:loss = 0.92369, step = 801 (0.137 sec)
INFO:tensorflow:global_step/sec: 811.92
INFO:tensorflow:loss = 0.905444,

In [31]:
# Score accuracy
test_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": X_test},
    y=y_test,
    num_epochs=1,
    shuffle=False)

In [32]:
ev = nn.evaluate(input_fn=test_input_fn)
print("Loss: %s" % ev["loss"])
print("Root Mean Squared Error: %s" % ev["rmse"])

INFO:tensorflow:Starting evaluation at 2017-11-24-00:24:35
INFO:tensorflow:Restoring parameters from /tmp/tmpc1j8auxb/model.ckpt-1280
INFO:tensorflow:Finished evaluation at 2017-11-24-00:24:35
INFO:tensorflow:Saving dict for global step 1280: global_step = 1280, loss = 1626.94, rmse = 40.4263
Loss: 1626.94
Root Mean Squared Error: 40.4263


In [33]:
for index, prediction_dictionary in enumerate(predictions):
    print(f"Prediction {index}, {prediction_dictionary['y']} ")

NameError: name 'predictions' is not defined